In [12]:
import pandas as pd

def get_id(dir_path='cmg-data/split-data', type='randomly'):
    with open(f'{dir_path}/{type}/train_id.txt') as file:
        train_id = [line.rstrip() for line in file]
    with open(f'{dir_path}/{type}/test_id.txt') as file:
        test_id = [line.rstrip() for line in file]
    return train_id, test_id

df = pd.read_parquet(f'cmg-data/cmg-data-processed.parquet', engine='fastparquet')
train_id, test_id = get_id(dir_path='cmg-data/split-data', type='cross_project')
train, test = df.loc[df['index'].isin(train_id)], df.loc[df['index'].isin(test_id)]

In [13]:
import nltk
from nltk import WordNetLemmatizer, pos_tag, WordPunctTokenizer, data
from nltk.corpus import wordnet
from tqdm import tqdm
import re

def write_string_to_file(absolute_filename, string):
    with open(absolute_filename, 'w') as fout:
        fout.write(string)

def word_tokenizer(sentence):
    words = WordPunctTokenizer().tokenize(sentence)
    return words

source_seqs = list()
target_seqs = list()
lang_seqs = list()

indexs = train['index'].unique()

for index in tqdm(indexs):
    df_commit = train[train['index']==index]
    diffs = list()
    langs = list()
    source_seq = ''
    for _, row in df_commit.iterrows():
        for l in row['diff'].splitlines():
            l = re.sub('@@.+?@@', '', l)
            l = re.sub(r'\s+', ' ', l)
            if len(l) <= 0:
                continue
            words = word_tokenizer(l)
            diffs.append(' '.join(words))
        type = row['new_path_file'].split('.')[-1]
        if type in ['c', 'h']:
            langs.append('c')
        else:
            langs.append('cpp')
        
        if row['old_path_file'] != None:
            old_f = word_tokenizer(row['old_path_file'])
            source_seq += 'mmm ' + ' '.join(old_f) + ' <nl> '
        
        if row['old_path_file'] != None:
            new_f = word_tokenizer(row['new_path_file'])
            source_seq += 'ppp ' + ' '.join(new_f) + ' <nl> '
        
        source_seq += ' <nl> '.join(diffs)
        label_words = row['label'].split()
        target_seq = ' '.join(label_words)
        # source_words = [word for word in source_seq.split()]
        # target_words = [word for word in target_seq.split()]
        
        # if len(source_words) > 100:
        #     continue

        # # if len(target_words) > 30 or not starts_with_verb(target_words):
        # #     continue
        # if len(target_words) > 30:
        #     continue
        
        # if cleaned and clean_msg(target_seq):
        #     continue

    source_seqs.append(source_seq)
    target_seqs.append(target_seq)
    lang_seqs.append(' '.join(langs))

write_string_to_file(f'CMG-data/cmg.train.diff', '\n'.join(source_seqs[:23172]))
write_string_to_file(f'CMG-data/cmg.train.msg', '\n'.join(target_seqs[:23172]))
write_string_to_file(f'CMG-data/cmg.train.lang', '\n'.join(lang_seqs[:23172]))

write_string_to_file(f'CMG-data/cmg.valid.diff', '\n'.join(source_seqs[23172:]))
write_string_to_file(f'CMG-data/cmg.valid.msg', '\n'.join(target_seqs[23172:]))
write_string_to_file(f'CMG-data/cmg.valid.lang', '\n'.join(lang_seqs[23172:]))

  0%|          | 0/25747 [00:00<?, ?it/s]

100%|██████████| 25747/25747 [01:19<00:00, 322.89it/s]


In [15]:
source_seqs = list()
target_seqs = list()
lang_seqs = list()

indexs = test['index'].unique()

for index in tqdm(indexs):
    df_commit = test[test['index']==index]
    diffs = list()
    langs = list()
    source_seq = ''
    for _, row in df_commit.iterrows():
        for l in row['diff'].splitlines():
            l = re.sub('@@.+?@@', '', l)
            l = re.sub(r'\s+', ' ', l)
            if len(l) <= 0:
                continue
            words = word_tokenizer(l)
            diffs.append(' '.join(words))
        type = row['new_path_file'].split('.')[-1]
        if type in ['c', 'h']:
            langs.append('c')
        else:
            langs.append('cpp')
        
        if row['old_path_file'] != None:
            old_f = word_tokenizer(row['old_path_file'])
            source_seq += 'mmm ' + ' '.join(old_f) + ' <nl> '
        
        if row['old_path_file'] != None:
            new_f = word_tokenizer(row['new_path_file'])
            source_seq += 'ppp ' + ' '.join(new_f) + ' <nl> '
        
        source_seq += ' <nl> '.join(diffs)
        label_words = row['label'].split()
        target_seq = ' '.join(label_words)
        # source_words = [word for word in source_seq.split()]
        # target_words = [word for word in target_seq.split()]
        
        # if len(source_words) > 100:
        #     continue

        # # if len(target_words) > 30 or not starts_with_verb(target_words):
        # #     continue
        # if len(target_words) > 30:
        #     continue
        
        # if cleaned and clean_msg(target_seq):
        #     continue

    source_seqs.append(source_seq)
    target_seqs.append(target_seq)
    lang_seqs.append(' '.join(langs))

write_string_to_file(f'CMG-data/cmg.test.diff', '\n'.join(source_seqs))
write_string_to_file(f'CMG-data/cmg.test.msg', '\n'.join(target_seqs))
write_string_to_file(f'CMG-data/cmg.test.lang', '\n'.join(lang_seqs))

  0%|          | 0/6406 [00:00<?, ?it/s]

100%|██████████| 6406/6406 [00:07<00:00, 884.14it/s] 


In [19]:
# Chuyen cmg vaild data thanh Race data
!python data_processing/preprocess.py

build: 100%|███████████████████████████████| 2575/2575 [00:22<00:00, 112.86it/s]
2575


In [21]:
# Chuyen cmg train data thanh Race data
!python data_processing/preprocess.py --diff_filename CMG-data/cmg.train.diff --msg_filename CMG-data/cmg.train.msg --lang_filename CMG-data/cmg.train.lang

build: 100%|██████████████████████████████| 23172/23172 [03:56<00:00, 98.03it/s]
23172


In [20]:
# Chuyen cmg test data thanh Race data
!python data_processing/preprocess.py --diff_filename CMG-data/cmg.test.diff --msg_filename CMG-data/cmg.test.msg --lang_filename CMG-data/cmg.test.lang

build: 100%|███████████████████████████████| 6406/6406 [00:55<00:00, 115.38it/s]
6406


In [22]:
!cp data_processing/saved_process/cmg.test.jsonl dataset/cpp/contextual_medits/test.jsonl
!cp data_processing/saved_process/cmg.train.jsonl dataset/cpp/contextual_medits/train.jsonl
!cp data_processing/saved_process/cmg.valid.jsonl dataset/cpp/contextual_medits/valid.jsonl

In [24]:
!bash run.sh cpp

saved_model/codet5/cpp/
============TRAINING============
^C


In [25]:
!python evalNMTS/eval.py --prd_dir evalNMTS/data/nngen.cmg.test.msg --gold_dir evalNMTS/data/cmg.test.msg 

predict lines:  6406
refs lines:  6406
EM = 8.57
precs = 25.392
recall = 25.431
Meteor: 10.698227282299198
ROUGE-L: 14.822569048428564
Total: 6406
Total: 6406
  bleu-4 = 16.618 
  bleu-normal = 16.618 
Bleu-B-Norm:  16.617827909354563
